In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py

# Helpful
import time
import datetime
import itertools
from itertools import product

# My Methods
import importlib
import CRPS
import EMOS
from CRPS import *
from EMOS import *

2023-05-15 12:36:28.504290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-15 12:36:28.504326: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

In [13]:
importlib.reload(CRPS)
importlib.reload(EMOS)

<module 'EMOS' from '/home/dchen/BA_CH_EN/EMOS.py'>

<IPython.core.display.Javascript object>

### Goal of this notebook: Implement EMOS and train it on train dataset

#### 1. Load dataset

In [9]:
t2m_train = xr.open_dataset("/Data/Delong_BA_Data/Mean_ens_std/t2m_train.h5")
t2m_X_train = t2m_train.t2m_train
t2m_y_train = t2m_train.t2m_truth

<IPython.core.display.Javascript object>

#### 2. Prepare Data

In [24]:
t2m_X_train_glob_mean = t2m_X_train.isel(phony_dim_4=0).values.flatten()
t2m_X_train_glob_std = t2m_X_train.isel(phony_dim_4=1).values.flatten()

<IPython.core.display.Javascript object>

In [31]:
t2m_y_train_glob_truth = t2m_y_train.values.flatten()

<IPython.core.display.Javascript object>

In [35]:
crps_baseline = crps_normal(
    mu=t2m_X_train_glob_mean, sigma=t2m_X_train_glob_std, y=t2m_y_train_glob_truth
)

/home/dchen/BA_CH_EN/CRPS.py:55: RuntimeWarning: invalid value encountered in true_divide
  loc = (y - mu) / sigma


<IPython.core.display.Javascript object>

In [41]:
np.isnan(crps_baseline).sum()

2139665

<IPython.core.display.Javascript object>

In [44]:
(t2m_X_train_glob_std == 0).sum()

2139665

<IPython.core.display.Javascript object>

#### 3. Train global Emos

In [15]:
EMOS_glob = build_EMOS_network_keras(compile=True)

<IPython.core.display.Javascript object>

In [17]:
EMOS_glob.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 1)            2           ['input_3[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 1)            2           ['input_4[0][0]']                
                                                                                            

<IPython.core.display.Javascript object>

In [34]:
EMOS_glob.fit(
    [t2m_X_train_glob_mean, t2m_X_train_glob_std],
    t2m_y_train_glob_truth,
    batch_size=12619,
    epochs=10,
    validation_split=0.2,
)

2023-05-15 14:30:01.660825: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2282741760 exceeds 10% of free system memory.
2023-05-15 14:30:02.640597: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2282741760 exceeds 10% of free system memory.
2023-05-15 14:30:03.609210: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2282741760 exceeds 10% of free system memory.
2023-05-15 14:30:04.692565: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4565483520 exceeds 10% of free system memory.
2023-05-15 14:30:05.803986: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4565483520 exceeds 10% of free system memory.


Epoch 1/10
45225/45225 [==============================] - 177s 3ms/step - loss: nan - val_loss: nan
Epoch 2/10
45225/45225 [==============================] - 148s 3ms/step - loss: nan - val_loss: nan
Epoch 3/10
 7489/45225 [===>..........................] - ETA: 1:49 - loss: nan

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [33]:
t2m_y_train_glob_truth.shape

(713356800,)

<IPython.core.display.Javascript object>